# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [1]:
# skopiuj pierścień ilorazowy wielomianów z pierwszych zajęć
# klasa z lekkimi modyfikacjami
import numpy as np

class ZnW:
    def __init__(self, N, W, p):
        self.N = N
        self.W = W
        self.r = self.calculate_polynomial(p)

    def calculate_polynomial(self, p):
        if not isinstance(p, np.ndarray):
            if isinstance(p, int):
                p = np.array([p])
            elif isinstance(p, list):
                p = np.array(p)
            else:
                raise TypeError(f"Argument powinien być obiektem klasy numpy.ndarray, listą lub liczbą całkowitą! Otrzymano typ: {type(p)}")

        _, remainder = np.polydiv(p, self.W)
        remainder_mod = []
        
        for coef in remainder:
            r = Zn(int(coef), self.N)
            remainder_mod.append(r.val)
            
        return np.array(remainder_mod)

    def check_argument(self, argument):
        if not isinstance(argument, ZnW):
            return ZnW(self.N, self.W, argument)
        elif argument.N != self.N:
            raise ValueError(f"Różne podstawy modularne N w argumentach: {self.N} oraz {argument.N}")
        elif not np.all(argument.W == self.W):
            raise ValueError(
                f"Różne wielomiany w pierścieniu ilorazowym: {self.polynomial_to_str(self.W)} oraz {self.polynomial_to_str(argument.W)}"
            )

        return argument

    def polynomial_to_str(self, poly):
        string = ""
        power = len(poly) - 1

        for i in range(len(poly)):
            coef = poly[i]
            
            if coef != 0:
                if i > 0: string += " + "
                if power - i > 1: string += f"{coef}x^{power - i}" if coef != 1 else f"x^{power - i}"
                elif power - i == 1: string += f"{coef}x" if coef != 1 else "x"
                else: string += f"{coef}"

        return string

    def __add__(self, other):
        other = self.check_argument(other)
        add_coeffs = np.polyadd(self.r, other.r)
        return ZnW(self.N, self.W, add_coeffs)

    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.check_argument(other)
        sub_coeffs = np.polysub(self.r, other.r)
        return ZnW(self.N, self.W, sub_coeffs)

    def __mul__(self, other):
        other = self.check_argument(other)
        mul_coeffs = np.polymul(self.r, other.r)
        return ZnW(self.N, self.W, mul_coeffs)

    def __rmul__(self, other):
        return self * other

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        return self.polynomial_to_str(self.r)

In [2]:
# potrzebuję też klasy pierścienia reszt modulo n - korzysta z niej powyższa klasa
class Zn:
    def __init__(self, val, N):
        self.val = val % N
        self.N = N

    def check_argument(self, argument):
        if not isinstance(argument, Zn):
            if isinstance(argument, int):
                return Zn(argument, self.N)
            raise TypeError(f"Argument operacji powinien być obiektem klasy Zn lub liczbą całkowitą! Otrzymano typ: {type(argument)}")
        elif argument.N != self.N:
            raise ValueError(f"Różne podstawy modularne N w argumentach: {self.N} oraz {argument.N}")

        return argument

    def is_negative_power(self, argument):
        return isinstance(argument, int) and argument < 0

    def find_modular_inverse(self):
        for i in range(1, self.N):
            if i * self.val % self.N == 1:
                return i

        raise ZeroDivisionError(f"{self.val} nie ma odwrotności modularnej dla podstawy modularnej {self.N}")
        
    def __eq__(self, other):
        return isinstance(other, Zn) and self.val == other.val and self.N == other.N
    
    def __add__(self, other):
        other = self.check_argument(other)
        return Zn(self.val + other.val, self.N)

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        other = self.check_argument(other)
        return Zn(self.val - other.val, self.N)

    def __rsub__(self, other):
        other = self.check_argument(other)
        return other - self

    def __mul__(self, other):
        other = self.check_argument(other)
        return Zn(self.val * other.val, self.N) 

    def __rmul__(self, other):
        return self * other

    def __pow__(self, other):
        if self.is_negative_power(other):
            power = -other
            mod_inv = self.find_modular_inverse()
            return Zn(mod_inv ** power, self.N)
        else:
            other = self.check_argument(other)
            return Zn(self.val ** other.val, self.N)

    def __rpow__(self, other):
        other = self.check_argument(other)
        return other ** self

    def __str__(self):
        return self.__repr__()
    
    def __repr__(self):
        return str(self.val)

## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

### Generowanie klucza

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A, t, s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

---

Najpierw przetestuję poprawność implementacji (zarówno tej funkcji, jak i klasy dla pierścienia wielomianowego) dla przykładowych, "zahardkodowanych" danych.

In [3]:
# parametry pierścienia zgodnie z poleceniem zadania
q = 17
W = np.array([1, 0, 0, 0, 1])

In [4]:
def key_gen_hardcoded():
    # przepisanie wartości testowych do obiektów klasy
    A = np.array([[ZnW(q, W, [6, 16, 16, 11]), ZnW(q, W, [9, 4, 6, 3])], 
                  [ZnW(q, W, [5, 3, 10, 1]), ZnW(q, W, [6, 1, 9, 15])]])
    s = np.array([ZnW(q, W, [-1, -1, 1, 0]), ZnW(q, W, [-1, 0, -1, 0])])
    e = np.array([ZnW(q, W, [1, 0, 0]), ZnW(q, W, [1, -1, 0])])
    
    t = A @ s + e
    
    return A, t, s

In [5]:
A, t, s = key_gen_hardcoded()

print(f"A = {A}\n")
print(f"t = {t}")
print(f"s = {s}")

A = [[6x^3 + 16x^2 + 16x + 11 9x^3 + 4x^2 + 6x + 3]
 [5x^3 + 3x^2 + 10x + 1 6x^3 + x^2 + 9x + 15]]

t = [16x^3 + 15x^2 + 7 10x^3 + 12x^2 + 11x + 6]
s = [16x^3 + 16x^2 + x 16x^3 + 16x]


### Szyfrowanie

Zaimplementuj funkcję `encrypt(A, t, m)` realizującą szyfrowanie w kryptosystemie Baby Kyber, gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych. 

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11), 8x^3+6x^2+9x+16)$

---

Podobnie jak poprzednio, przetestuję działanie funkcji dla sztywno przyjętych danych testowych.

In [6]:
# jako że mamy dane q = 17, to hardkoduję sufit z q/2 - czyli 9
q2_ceil = 9

In [7]:
def encrypt_hardcoded(A, t, m):
    # przepisanie wartości testowych do obiektów klasy
    r = np.array([ZnW(q, W, [-1, 1, 0, 0]), ZnW(q, W, [1, 1, 0, -1])])
    e1 = np.array([ZnW(q, W, [1, 1, 0]), ZnW(q, W, [1, 0, 0])])
    e2 = np.array([ZnW(q, W, [-1, -1, 0, 0])])

    u = A.T @ r + e1
    v = (np.array([t.T @ r]) + e2 + q2_ceil * ZnW(q, W, m))[0]
    
    return (u, v)

In [8]:
# przykładowa, testowana wiadomość w postaci listy
m = [1, 0, 1, 1]

c = encrypt_hardcoded(A, t, m)

print("c = (u, v)\n")
print(f"u = {c[0]}")
print(f"v = {c[1]}")

c = (u, v)

u = [11x^3 + 11x^2 + 10x + 3 4x^3 + 4x^2 + 13x + 11]
v = 8x^3 + 6x^2 + 9x + 16


### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c, s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


In [9]:
def decrypt(c, s):
    u, v = c
    mn = v - s.T @ u
    
    # trochę "naokoło", ale poprawnie zapisany warunek dla "zaokrąglania" współczynników
    mn = np.where(np.abs(mn.r - q2_ceil) < q2_ceil / 2, 1, 0)
    
    # zwracam również listę
    return list(mn)

In [10]:
mn = decrypt(c, s)

print(f"Odszyfrowana wiadomość mn = {ZnW(q, W, mn)}")

Odszyfrowana wiadomość mn = x^3 + x + 1



---

Wygląda na to, że implementacja jest poprawna. Teraz przejdę do implementacji funkcji działających dla ogólnego przypadku:
1. `key_gen()`
2. `encrypt(A, t, m)`

Powyższa funkcja `decrypt(c, s)` nie wymaga drugiej "wersji". Zgodnie z teorią, współczynniki wielomianów w $A$ są generowane losowo z rozkładu jednostajnego (z zakresu $\{0, 1, \dots q-1\}$). Z kolei pozostałe elementy, mianowicie $s$ i $e$ w generowaniu klucza oraz $r$, $e_1$ i $e_2$ przy szyfrowaniu pochodzą z rozkładu $\beta$, który tutaj oznacza tak naprawdę odpowiednio ważony rozkład ze zbioru $\{-1,0,1\}$, przy czym wagi wynoszą odpowiednio 0.1, 0.8 i 0.1.

In [11]:
def beta(size=4):
    return np.random.choice(a=[-1, 0, 1], size=size, p=[0.1, 0.8, 0.1])

W przypadku wektorów pochodzących z rozkładu $\beta$ nie chcemy dopuścić do sytuacji, w której __wszystkie__ elementy tego wektora (wielomiany o stopniu co najwyżej 3) mają __wszystkie__ współczynniki zerowe (ogólnie mogą wystąpić takie wielomiany, ale musi być jakiś wielomian niezerowy). Poniższa funkcja zapobiega takim sytuacjom.

In [12]:
def verify(array):
    for element in array:
        if not np.all(element == 0):
            return True
    
    return False

In [13]:
def key_gen():
    A = np.array([[ZnW(q, W, np.random.randint(low=0, high=q, size=4)), ZnW(q, W, np.random.randint(low=0, high=q, size=4))], 
                  [ZnW(q, W, np.random.randint(low=0, high=q, size=4)), ZnW(q, W, np.random.randint(low=0, high=q, size=4))]])
    
    s = np.array([ZnW(q, W, beta()), ZnW(q, W, beta())])
    while not verify(s): s = np.array([ZnW(q, W, beta()), ZnW(q, W, beta())])
    
    e = np.array([ZnW(q, W, beta()), ZnW(q, W, beta())])
    while not verify(e): e = np.array([ZnW(q, W, beta()), ZnW(q, W, beta())])
    
    t = A @ s + e
    
    return A, t, s

In [14]:
def encrypt(A, t, m):
    r = np.array([ZnW(q, W, beta()), ZnW(q, W, beta())])
    while not verify(r): r = np.array([ZnW(q, W, beta()), ZnW(q, W, beta())])
    
    e1 = np.array([ZnW(q, W, beta()), ZnW(q, W, beta())])
    while not verify(e1): e1 = np.array([ZnW(q, W, beta()), ZnW(q, W, beta())])
    
    e2 = np.array([ZnW(q, W, beta())])
    while not verify(e2): e2 = np.array([ZnW(q, W, beta())])

    u = A.T @ r + e1
    v = (np.array([t.T @ r]) + e2 + q2_ceil * ZnW(q, W, m))[0]
    
    return (u, v)

### Testy

In [15]:
import secrets as sc

success = 0

for i in range(1000):
    output = []
    A, t, s = key_gen()
    
    m = [sc.choice((0, 1)) for k in range(4)]
    
    c = encrypt(A, t, m)
    m_n = decrypt(c, s)

    if m_n == m:
        success += 1

print(f'Success rate: {success * 100 / 1000}%')

Success rate: 74.3%
